In [1]:
import duckdb

con = duckdb.connect(database="data/scd_2.duckdb")

#  scd creation
con.execute("""
    create or replace table scd as (
         select 1 id, date'2022-06-01' effective_from, date'9999-12-31' effective_to, timestamp'2023-01-01' snapshot_update_ts union all
         select 1, date'2022-05-26', date'2022-05-31', timestamp'2023-01-01' union all
         select 1, date'2022-05-24', date'2022-05-25', timestamp'2023-01-01' union all
         select 1, date'2022-05-15', date'2022-05-23', timestamp'2023-01-01' union all
         select 1, date'2022-05-01', date'2022-05-14', timestamp'2023-01-01' union all
         select 1, date'2022-04-15', date'2022-04-30', timestamp'2023-01-01' union all
         select 1, date'2022-04-01', date'2022-04-14', timestamp'2023-01-01' union all
         select 1, date'2022-03-15', date'2022-03-31', timestamp'2023-01-01' union all
         select 2, date'2022-01-15', date'9999-12-31', timestamp'2023-01-01' union all
         select 3, date'2022-01-15', date'9999-12-31', timestamp'2023-01-01'
    );
""")
#   delta creation 
con.execute("""
    create or replace table delta as (
         select 1 id, date'2022-06-15' fact_date union all
         select 1, date'2022-06-01' union all
         select 1, date'2022-05-30' union all
         select 1, date'2022-05-30' union all
         select 1, date'2022-05-28' union all
         select 1, date'2022-05-20' union all
         select 1, date'2022-05-13' union all
         select 1, date'2022-05-13' union all
         select 1, date'2022-04-01' union all
         select 1, date'2022-03-01'
    );
""")

In [2]:
# insert data
# con.execute("""
#     insert into scd
#     select 1, v::date-1, v::date, current_timestamp
#     from generate_series(date'2022-03-14', date'2022-03-04', interval '-1 days') t(v);
# """)

In [3]:
con.execute("""
    create or replace table scd_final as with
     combine as (
      select * from delta
      union all
      select * exclude (effective_to, snapshot_update_ts)
      from scd
     ),
     dedupe as (
      select * from combine
      qualify row_number() over (partition by sha1(to_json((
       id, fact_date
      ))) order by fact_date) = 1
     )
     select
      id,
      fact_date as effective_from,
      ifnull(lead(fact_date) over (
       partition by sha1(to_json((
        id
       ))) order by fact_date) - 1,
       date'9999-12-31'
      ) as effective_to,
      current_timestamp as snapshot_update_ts
     from dedupe
;
""")